In [13]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

In [52]:
pd_net = pd.read_csv('network_parent_child.csv')

In [51]:
display(pd_net)

,geo_id,parent_id,lon,lat,device_type
0,950351437,950351423.0,-83.759581,32.658571,tap
1,953550857,953550852.0,-83.586641,32.539248,tap
2,933436770,933436754.0,-83.713820,32.831317,tap
3,950348326,950348313.0,-83.754149,32.656190,tap
4,931035183,931035168.0,-83.690723,32.874315,tap
...,...,...,...,...,...
206061,931033358,931033305.0,-83.695732,32.879170,address
206062,948424286,948424277.0,-83.663388,32.791191,address
206063,932103961,932103047.0,-83.645818,32.636752,address
206064,932434238,932434221.0,-83.742775,32.916636,address


In [27]:
len(pd_net.geo_id.unique())

206066

In [10]:
pd_net.geo_id.unique()

array([950351437, 953550857, 933436770, ..., 932103961, 932434238,
       952074725], dtype=int64)

In [24]:
testloc = pd_net[['lat','lon']].to_numpy()[0].tolist()

In [47]:
pd_net.groupby('device_type').count()

,geo_id,parent_id,lon,lat
device_type,,,,
address,86800,86800,86798,86798
amplifier,14194,14194,14194,14194
coaxial_splitter,9929,9929,9929,9929
cox_usps_address,25822,25786,25822,25822
in_line_equalizer,3212,3212,3212,3212
internal_coupler,1157,1157,1157,1157
optical_node,449,0,449,449
power_block,1169,1169,1169,1169
power_inserter,1073,1073,1073,1073


In [45]:
pd_net[pd_net.device_type == 'tap']

,geo_id,parent_id,lon,lat,device_type
0,950351437,950351423.0,-83.759581,32.658571,tap
1,953550857,953550852.0,-83.586641,32.539248,tap
2,933436770,933436754.0,-83.713820,32.831317,tap
3,950348326,950348313.0,-83.754149,32.656190,tap
4,931035183,931035168.0,-83.690723,32.874315,tap
...,...,...,...,...,...
201493,930837505,930837494.0,-83.689440,32.755609,tap
201496,932406050,932406031.0,-83.671313,32.822743,tap
201497,931098653,931098633.0,-83.762816,32.910869,tap
201500,931940887,931940870.0,-83.714541,32.625849,tap


In [49]:
pd_net_plot.merge(pd_net, left_on='geo_id', right_on='parent_id', how='inner')

,geo_id_x,parent_id_x,lon_x,lat_x,device_type_x,geo_id_y,parent_id_y,lon_y,lat_y,device_type_y
0,932033367,932033357.0,-83.640841,32.598431,power_block,932033411,932033367.0,-83.640841,32.598431,amplifier
1,932436576,932436567.0,-83.745769,32.904414,power_block,932436611,932436576.0,-83.745769,32.904414,power_inserter
2,932087589,932087582.0,-83.682011,32.602991,power_block,932087600,932087589.0,-83.682168,32.602877,power_block
3,950259838,950259828.0,-83.779138,32.832067,power_block,950259860,950259838.0,-83.779138,32.832067,amplifier
4,933044407,933044394.0,-83.734554,32.813802,power_block,933044416,933044407.0,-83.736256,32.812956,power_block
...,...,...,...,...,...,...,...,...,...,...
1141,948432347,948432341.0,-83.689501,32.904327,power_block,948432352,948432347.0,-83.688044,32.905065,power_block
1142,932446876,932446867.0,-83.745374,32.832272,power_block,932446904,932446876.0,-83.745374,32.832272,amplifier
1143,950207267,950207223.0,-83.642828,32.618158,power_block,950207276,950207267.0,-83.644939,32.619674,power_block
1144,930664761,930663217.0,-83.671312,32.808627,power_block,930664770,930664761.0,-83.671312,32.808627,coaxial_splitter


In [55]:
my_map = folium.Map(location = testloc, zoom_start = 13)
feature_group_pt = folium.FeatureGroup("Locations")
feature_group_pl = folium.FeatureGroup("Connections")

pd_net_pt_plot = pd_net[pd_net.device_type == 'optical_node']

for lat, lng, name in pd_net_pt_plot[['lat','lon','device_type']].to_numpy():
    feature_group_pt.add_child(folium.Marker(location=[lat,lng],popup=name))

pd_net_pl_plot = pd_net_pt_plot.merge(pd_net, left_on='geo_id', right_on='parent_id', how='inner')

for lat_x, lng_x, name, lat_y, lng_y in pd_net_pl_plot[['lat_x','lon_x','device_type_x','lat_y','lon_y']].to_numpy():
    feature_group_pl.add_child(folium.vector_layers.PolyLine(locations=[(lat_x,lng_x),(lat_y,lng_y)]))

my_map.add_child(feature_group_pt)
my_map.add_child(feature_group_pl)
my_map